In [1]:
import os
import time

import numpy as np
import torch

import onnx
import onnxruntime as ort
from onnxruntime import InferenceSession

import warnings

import argparse

from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer, 
                          Phi3ForCausalLM,
                          Phi3Config,
                          pipeline,
                          )
from transformers.pipelines.pt_utils import KeyDataset
from huggingface_hub import hf_hub_download
import torch_tensorrt 
import utils
import torch.backends.cudnn as cudnn
device = torch.device( "cuda" if torch.cuda.is_available() else cpu )

In [2]:
# 특정 경고 무시
warnings.filterwarnings("ignore", category=torch.jit.TracerWarning)

In [3]:
parser = argparse.ArgumentParser(description="TensorRT")
parser.add_argument("--model_path", type=str, default="models/")
parser.add_argument("--model_name", type=str, default="Phi-3-medium-4k-instruct")
parser.add_argument("--data_path", type=str, default="data/")
parser.add_argument("--dataset", type=str, default="test_dataset.jsonl")
parser.add_argument('--seed',type=int, default=0)
parser.add_argument("--batch_size", type=int, default=1)
parser.add_argument('--dtype',type=str, default="auto")
parser.add_argument('--n', type=int, default=1)
parser.add_argument('--temperature', type=float, default=0.0)

config = parser.parse_args([])

utils.seed_everything(config.seed)
CURR_PATH = os.getcwd()

In [4]:
# 1. 모델 및 토크나이저 로드
model_id = os.path.join(CURR_PATH, config.model_path, config.model_name)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

max_memory = {
    0: "28GB",  # GPU 0 
    'cpu': "28GB"  # CPU 
}

llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    max_memory = max_memory 
)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

RuntimeError: NVML_SUCCESS == r INTERNAL ASSERT FAILED at "../c10/cuda/CUDACachingAllocator.cpp":838, please report a bug to PyTorch. 

In [ ]:
# GPU 메모리 캐시 초기화
torch.cuda.empty_cache()

# CUDA 캐싱 할당기에서 할당된 모든 메모리 해제
torch.cuda.reset_peak_memory_stats()

In [ ]:
# ONNX 변환 시 사용할 더미 입력 데이터 생성
# 시퀀스 길이는 모델에 맞게 조정할 수 있습니다 (예: 828)
dummy_input = torch.randint(0, tokenizer.vocab_size, (1, 828), dtype=torch.long).to("cpu")

# 모델을 평가 모드로 설정 (Dropout 등 비활성화)
llm.eval()

# 모델을 CPU로 이동
llm = llm.to('cpu')

# ONNX 파일로 저장할 경로 설정
onnx_model_path = "./onnx/phi3-medium-4k-instruct-cuda-fp16.onnx"

# ONNX로 변환
torch.onnx.export(
    llm,                           # 변환할 모델
    dummy_input,                   # 더미 입력 데이터
    onnx_model_path,               # 저장할 ONNX 파일 경로
    input_names=["input_ids"],      # 입력 텐서 이름
    output_names=["generated_outputs"],        # 출력 텐서 이름
    dynamic_axes={                  # 시퀀스 길이와 배치 크기를 동적으로 처리
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "generated_outputs": {0: "batch_size", 1: "sequence_length"}
    },
    opset_version=13,               # ONNX opset 버전 설정 (최신 PyTorch에서는 13 권장)
    do_constant_folding=True,       # 상수 접기 최적화
)

print(f"ONNX 모델이 {onnx_model_path}에 저장되었습니다.")

  if input_shape[-1] > 1 or self.sliding_window is not None:

  if past_key_values_length > 0:

  if attn_weights.size() != (bsz, self.num_heads, q_len, kv_seq_len):

  if attention_mask.size() != (bsz, 1, q_len, kv_seq_len):

  if attn_output.size() != (bsz, self.num_heads, q_len, self.head_dim):

